## HPCCG

clang++-5.0.1 -g -O3 -funroll-loops -march=native    
./test_HPCCG 100 100 100

#### Serial Run
(Haswell) Intel(R) Xeon(R) CPU E5-2699 v3 @ 2.30GHz  
`mulsd`: 5 cycles  
`movsd`: 3 cycles  
L1 Cache: 32 kB, 8 way, 64 sets, 64 B line size, **latency 4**, per core.    
L2 Cache: 256 kB, 8 way, 512 sets, 64 B line size, **latency 12**, per core.

|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | 
|:---|:---:|:---:|:---:|
|`HPC_sparsemv( )` | 80.6% CPU |    1.04 Ins / Cycle    | 7.5% L1 Data Cache Miss |
```c
|66|int HPC_sparsemv( HPC_Sparse_Matrix *A,                  
|67|		 const double * const x, double * const y)
|68|{
|69|
|70|  const int nrow = (const int) A->local_nrow;
|71|
|72|#ifdef USING_OMP
|73|#pragma omp parallel for
|74|#endif
|75|  for (int i=0; i< nrow; i++)
|76|    {
|77|      double sum = 0.0;
|78|      const double * const cur_vals = 
|79|     (const double * const) A->ptr_to_vals_in_row[i];
|80|
|81|      const int    * const cur_inds = 
|82|     (const int    * const) A->ptr_to_inds_in_row[i];
|83|
|84|      const int cur_nnz = (const int) A->nnz_in_row[i];
|85|
|86|      for (int j=0; j< cur_nnz; j++)                     //  9.9% CPUTIME(E)
```
|Scope|  CPUTIME  | Instructions per Cycle | L1 Data Cache Miss Rate | 
|:---|:---:|:---:|:---:|
|`sum += cur_vals[j]*x[cur_inds[j]]; ` | **58.8% CPU** |    1.09 Ins / Cycle    | 7.6% L1 Data Cache Miss |
```c
|87|        sum += cur_vals[j]*x[cur_inds[j]];             
|88|      y[i] = sum;
|89|    }
|90|  return(0);
|91|}
```

#### Memory Latency Bound by `x[cur_inds[j]]`

---

---

#### CPI  
.96 Cycles per Instruction in `HPC_sparsemv`.  
.91 Cycles per Instruction on line 87. 
#### Issue Cycles
`HPC_sparsemv`:  
-- 3.63e+09 Full Issue | 30.3% Cycles Issuing Max Instructions  
-- 1.80e+07 No Issue   |  0.2% Cycles Issuing No Instructions  
-- 1.20e+10 Total Cycles   
line `87`:  
-- 1.97e+09 Full Issue | 22.5% Cycles Issuing Max Instructions  
-- 1.00e+07 No Issue   |  0.1% Cycles Issuing No Instructions  
-- 8.77e+09 Total Cycles   
#### Retiring Cycles  
`HPC_sparsemv`:   
-- 5.77e+09 Full Retire | 48.1% Cycles Retiring Max Instructions  
-- 2.83e+09 No Retire   | 23.6% Cycles Retiring No Instructions  
-- 1.20e+10 Total Cycles  
line `87`:  
-- 4.46e+09 Full Retire | 50.9% Cycles Retiring Max Instructions  
-- 1.99e+09 No Retire   | 22.7% Cycles Retiring No Instructions  
-- 8.77e+09 Total Cycles   

### Memory  
#### Data Cache  
`HPC_sparsemv`:  
-- 9.50e+08 L1 Data Cache Misses | 7.5% L1 Cache Miss Rate   
-- 3.75e+08 L2 Data Cache Misses | 60.5% L1 Misses Hit L2  
-- 1.26e+10 Load/Store Instructions    
line `87`:   
-- 7.18e+08 L1 Data Cache Misses | 7.6% L1 Cache Miss Rate  
-- 2.84e+08 L2 Data Cache Misses | 60.5% L1 Misses Hit L2  
-- 9.41e+09 Load/Store Instructions

In [25]:
1 - (.284 / .718)

0.6044568245125348

In [4]:
9.55 / 8.77

1.088939566704675

```
BE             Backend_Bound:                                      54.44 % Slots  [  4.76%]
BE/Mem         Backend_Bound.Memory_Bound:                         21.91 % Slots  [  4.76%]
BE/Core        Backend_Bound.Core_Bound:                           32.54 % Slots  [  4.76%]
BE/Mem         Backend_Bound.Memory_Bound.L1_Bound:                11.41 % Stalls [  4.76%]
        This metric estimates how often the CPU was stalled without
        loads missing the L1 data cache...
        Sampling events:  mem_load_uops_retired.l1_hit:pp mem_load_uops_retired.hit_lfb:pp
BE/Core        Backend_Bound.Core_Bound.Ports_Utilization:         26.51 % Clocks [  4.76%] <==
        This metric estimates cycles fraction the CPU performance
        was potentially limited due to Core computation issues (non
        divider-related)...
               MUX:                                                 4.76 %       
        PerfMon Event Multiplexing accuracy indicator

```
